# HF Transformers Core Model：Pipelines

In [54]:
import warnings
from transformers import logging
# 忽略所有警告
warnings.filterwarnings('ignore')

# 设置transformers库的日志级别为ERROR
logging.set_verbosity_error()

## Natural Language Processing(NLP)

### Text Classification

In [33]:
from transformers import pipeline

models=[
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
]
texts = [
    "Today Shanghai is really cold.",
    "I think the taste of the garlic mashed pork in this store is average.",
    "You learn things really quickly. You understand the theory class as soon as it is taught.",
    "今儿上海可真冷啊",
    "我觉得这家店蒜泥白肉的味道一般",
    "你学东西真的好快，理论课一讲就明白了"
]
pipes = [pipeline(task="sentiment-analysis",model=model) for model in models]
for pipe in pipes:
    print(f"{pipe.model.name_or_path}:",*pipe(texts), sep="\n\t")

distilbert/distilbert-base-uncased-finetuned-sst-2-english:
	{'label': 'NEGATIVE', 'score': 0.9995032548904419}
	{'label': 'NEGATIVE', 'score': 0.9984821677207947}
	{'label': 'POSITIVE', 'score': 0.9961802959442139}
	{'label': 'NEGATIVE', 'score': 0.8957217335700989}
	{'label': 'NEGATIVE', 'score': 0.9238731265068054}
	{'label': 'NEGATIVE', 'score': 0.8578692674636841}
lxyuan/distilbert-base-multilingual-cased-sentiments-student:
	{'label': 'negative', 'score': 0.7824518084526062}
	{'label': 'negative', 'score': 0.37749963998794556}
	{'label': 'positive', 'score': 0.7639099359512329}
	{'label': 'negative', 'score': 0.6657698154449463}
	{'label': 'neutral', 'score': 0.6030055284500122}
	{'label': 'positive', 'score': 0.9461327791213989}


### Token Classification

In [79]:
from transformers import pipeline

models=[
    "dbmdz/bert-large-cased-finetuned-conll03-english",
    "dslim/bert-base-NER"
]
# 推理函数
def classifier(text,grouped=False):
    classifiers = [pipeline(task="ner",model=model,grouped_entities=grouped) for model in models]
    for classifier in classifiers:
        preds = classifier(text)
        print(f"{classifier.model.name_or_path}:",*preds, sep="\n\t")

In [80]:
classifier("Hugging Face is a French company based in New York City.")

dbmdz/bert-large-cased-finetuned-conll03-english:
	{'entity': 'I-ORG', 'score': 0.9967675, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
	{'entity': 'I-ORG', 'score': 0.92930275, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
	{'entity': 'I-ORG', 'score': 0.9763208, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
	{'entity': 'I-MISC', 'score': 0.99828726, 'index': 6, 'word': 'French', 'start': 18, 'end': 24}
	{'entity': 'I-LOC', 'score': 0.99896204, 'index': 10, 'word': 'New', 'start': 42, 'end': 45}
	{'entity': 'I-LOC', 'score': 0.9986792, 'index': 11, 'word': 'York', 'start': 46, 'end': 50}
	{'entity': 'I-LOC', 'score': 0.9992418, 'index': 12, 'word': 'City', 'start': 51, 'end': 55}
dslim/bert-base-NER:
	{'entity': 'B-ORG', 'score': 0.8935039, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
	{'entity': 'I-ORG', 'score': 0.91499877, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
	{'entity': 'I-ORG', 'score': 0.97772026, 'index': 3, 'word': 'Face', 'start': 8, 'end': 1

In [81]:
classifier("Hugging Face is a French company based in New York City.",True)

dbmdz/bert-large-cased-finetuned-conll03-english:
	{'entity_group': 'ORG', 'score': 0.9674637, 'word': 'Hugging Face', 'start': 0, 'end': 12}
	{'entity_group': 'MISC', 'score': 0.99828726, 'word': 'French', 'start': 18, 'end': 24}
	{'entity_group': 'LOC', 'score': 0.99896103, 'word': 'New York City', 'start': 42, 'end': 55}
dslim/bert-base-NER:
	{'entity_group': 'ORG', 'score': 0.928741, 'word': 'Hugging Face', 'start': 0, 'end': 12}
	{'entity_group': 'MISC', 'score': 0.9996295, 'word': 'French', 'start': 18, 'end': 24}
	{'entity_group': 'LOC', 'score': 0.9994915, 'word': 'New York City', 'start': 42, 'end': 55}


### Question Answering

In [76]:
from transformers import pipeline

models=[
    "distilbert/distilbert-base-cased-distilled-squad",
    "deepset/roberta-base-squad2"
]
question_answerers = [{'model':model, 'pipe':pipeline(task="question-answering",model=model)} for model in models]
# 推理函数
def questions(question,context):
    for item in question_answerers:
        model=item['model']
        pipe=item['pipe']
        pred = pipe(question=question, context=context)
        print(f"{model}:\n\t{pred}")

In [77]:
questions(
    "What is the name of the repository?",
    "The name of the repository is huggingface/transformers"
)

distilbert/distilbert-base-cased-distilled-squad:
	{'score': 0.9327335953712463, 'start': 30, 'end': 54, 'answer': 'huggingface/transformers'}
deepset/roberta-base-squad2:
	{'score': 0.906841516494751, 'start': 30, 'end': 54, 'answer': 'huggingface/transformers'}


In [78]:
questions(
    "What is the capital of China?", 
    "On 1 October 1949, CCP Chairman Mao Zedong formally proclaimed the People's Republic of China in Tiananmen Square, Beijing."
)

distilbert/distilbert-base-cased-distilled-squad:
	{'score': 0.9458329677581787, 'start': 115, 'end': 122, 'answer': 'Beijing'}
deepset/roberta-base-squad2:
	{'score': 0.7503766417503357, 'start': 115, 'end': 122, 'answer': 'Beijing'}


### Summarization

In [87]:
from transformers import pipeline

models=[
    "google-t5/t5-base",
    "facebook/bart-large-cnn"
]
pipes = [{'model':model, 'pipe':pipeline(task="summarization",model=model,min_length=8,max_length=32)} for model in models]
# 推理函数
def summarizer(text):
    for item in pipes:
        model=item['model']
        pipe=item['pipe']
        pred = pipe(text)
        print(f"{model}:\n\t{pred}")

In [88]:
summarizer(
    """
    In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, 
    replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention. 
    For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. 
    On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. 
    In the former task our best model outperforms even all previously reported ensembles.
    """
)

google-t5/t5-base:
	[{'summary_text': 'the Transformer is the first sequence transduction model based entirely on attention . it replaces recurrent layers commonly used in encoder-decode'}]
facebook/bart-large-cnn:
	[{'summary_text': 'The Transformer is the first sequence transduction model based entirely on attention. It replaces the recurrent layers most commonly used in encoder-decoder'}]


In [89]:
summarizer(
    '''
    Large language models (LLM) are very large deep learning models that are pre-trained on vast amounts of data. 
    The underlying transformer is a set of neural networks that consist of an encoder and a decoder with self-attention capabilities. 
    The encoder and decoder extract meanings from a sequence of text and understand the relationships between words and phrases in it.
    Transformer LLMs are capable of unsupervised training, although a more precise explanation is that transformers perform self-learning. 
    It is through this process that transformers learn to understand basic grammar, languages, and knowledge.
    Unlike earlier recurrent neural networks (RNN) that sequentially process inputs, transformers process entire sequences in parallel. 
    This allows the data scientists to use GPUs for training transformer-based LLMs, significantly reducing the training time.
    '''
)

google-t5/t5-base:
	[{'summary_text': 'large language models (LLMs) are very large deep learning models pre-trained on vast amounts of data . transformers are capable of un'}]
facebook/bart-large-cnn:
	[{'summary_text': 'Transformers are a set of neural networks that consist of an encoder and a decoder with self-attention capabilities. Unlike earlier recurrent neural'}]


## Audio

### Audio classification

In [14]:
from transformers import pipeline

models=[
    "superb/hubert-base-superb-er",
    "MIT/ast-finetuned-audioset-10-10-0.4593"
]
pipes = [{'model':model, 'pipe':pipeline(task="audio-classification",model=model)} for model in models]
# 推理函数
def classifier(text):
    for item in pipes:
        model=item['model']
        pipe=item['pipe']
        preds = pipe(text)
        preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
        print(f"{model}:",*preds, sep="\n\t")

In [15]:
classifier("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

superb/hubert-base-superb-er:
	{'score': 0.4532, 'label': 'hap'}
	{'score': 0.3622, 'label': 'sad'}
	{'score': 0.0943, 'label': 'neu'}
	{'score': 0.0903, 'label': 'ang'}
MIT/ast-finetuned-audioset-10-10-0.4593:
	{'score': 0.4208, 'label': 'Speech'}
	{'score': 0.1793, 'label': 'Rain on surface'}
	{'score': 0.1301, 'label': 'Rain'}
	{'score': 0.096, 'label': 'Raindrop'}
	{'score': 0.0578, 'label': 'Music'}


In [16]:
classifier("data/audio/mlk.flac")

superb/hubert-base-superb-er:
	{'score': 0.4532, 'label': 'hap'}
	{'score': 0.3622, 'label': 'sad'}
	{'score': 0.0943, 'label': 'neu'}
	{'score': 0.0903, 'label': 'ang'}
MIT/ast-finetuned-audioset-10-10-0.4593:
	{'score': 0.4208, 'label': 'Speech'}
	{'score': 0.1793, 'label': 'Rain on surface'}
	{'score': 0.1301, 'label': 'Rain'}
	{'score': 0.096, 'label': 'Raindrop'}
	{'score': 0.0578, 'label': 'Music'}


### Automatic speech recognition（ASR）

In [22]:
from transformers import pipeline

models=[
    "openai/whisper-small",
    "facebook/wav2vec2-base-960h"
]
pipes = [{'model':model, 'pipe':pipeline(task="automatic-speech-recognition",model=model)} for model in models]
# 推理函数
def recognition(text):
    for item in pipes:
        model=item['model']
        pipe=item['pipe']
        pred = pipe(text)
        print(f"{model}:\n\t{pred}")

In [23]:
recognition("data/audio/mlk.flac")

openai/whisper-small:
	{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'}
facebook/wav2vec2-base-960h:
	{'text': 'I HAVE A DREAM BUT ONE DAY THIS NATION WILL RISE UP LIVE UP THE TRUE MEANING OF ITS TREES'}


## Computer Vision

### Image Classificaiton

In [35]:
from transformers import pipeline

models=[
    "google/vit-base-patch16-224",
    "microsoft/resnet-50"
]
pipes = [{'model':model, 'pipe':pipeline(task="image-classification",model=model)} for model in models]
# 推理函数
def classifier(text):
    for item in pipes:
        model=item['model']
        pipe=item['pipe']
        preds = pipe(text)
        preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
        print(f"{model}:",*preds, sep="\n\t")

In [36]:
classifier("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

google/vit-base-patch16-224:
	{'score': 0.4335, 'label': 'lynx, catamount'}
	{'score': 0.0348, 'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'}
	{'score': 0.0324, 'label': 'snow leopard, ounce, Panthera uncia'}
	{'score': 0.0239, 'label': 'Egyptian cat'}
	{'score': 0.0229, 'label': 'tiger cat'}
microsoft/resnet-50:
	{'score': 0.5874, 'label': 'lynx, catamount'}
	{'score': 0.1289, 'label': 'tabby, tabby cat'}
	{'score': 0.075, 'label': 'marmot'}
	{'score': 0.0382, 'label': 'badger'}
	{'score': 0.0131, 'label': 'Egyptian cat'}


![](data/image/cat-chonk.jpeg)

In [37]:
classifier("data/image/cat-chonk.jpeg")

google/vit-base-patch16-224:
	{'score': 0.4335, 'label': 'lynx, catamount'}
	{'score': 0.0348, 'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'}
	{'score': 0.0324, 'label': 'snow leopard, ounce, Panthera uncia'}
	{'score': 0.0239, 'label': 'Egyptian cat'}
	{'score': 0.0229, 'label': 'tiger cat'}
microsoft/resnet-50:
	{'score': 0.5874, 'label': 'lynx, catamount'}
	{'score': 0.1289, 'label': 'tabby, tabby cat'}
	{'score': 0.075, 'label': 'marmot'}
	{'score': 0.0382, 'label': 'badger'}
	{'score': 0.0131, 'label': 'Egyptian cat'}


![](data/image/panda.jpg)

In [38]:
classifier("data/image/panda.jpg")

google/vit-base-patch16-224:
	{'score': 0.9962, 'label': 'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca'}
	{'score': 0.0018, 'label': 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens'}
	{'score': 0.0002, 'label': 'ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus'}
	{'score': 0.0001, 'label': 'sloth bear, Melursus ursinus, Ursus ursinus'}
	{'score': 0.0001, 'label': 'brown bear, bruin, Ursus arctos'}
microsoft/resnet-50:
	{'score': 0.9768, 'label': 'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca'}
	{'score': 0.0088, 'label': 'indri, indris, Indri indri, Indri brevicaudatus'}
	{'score': 0.0004, 'label': 'groenendael'}
	{'score': 0.0003, 'label': 'Siberian husky'}
	{'score': 0.0002, 'label': 'malamute, malemute, Alaskan malamute'}


### Object Detection

In [43]:
from transformers import pipeline

models=[
    "facebook/detr-resnet-50",
    "SenseTime/deformable-detr"
]
pipes = [{'model':model, 'pipe':pipeline(task="object-detection",model=model)} for model in models]
# 推理函数
def detector(text):
    for item in pipes:
        model=item['model']
        pipe=item['pipe']
        preds = pipe(text)
        preds = [{"score": round(pred["score"], 4), "label": pred["label"], "box": pred["box"]} for pred in preds]
        print(f"{model}:",*preds, sep="\n\t")

In [44]:
detector("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

facebook/detr-resnet-50:
	{'score': 0.9864, 'label': 'cat', 'box': {'xmin': 178, 'ymin': 154, 'xmax': 882, 'ymax': 598}}
SenseTime/deformable-detr:
	{'score': 0.8068, 'label': 'cat', 'box': {'xmin': 179, 'ymin': 154, 'xmax': 877, 'ymax': 595}}


![](data/image/cat_dog.jpg)

In [46]:
detector("data/image/cat_dog.jpg")

facebook/detr-resnet-50:
	{'score': 0.9985, 'label': 'cat', 'box': {'xmin': 78, 'ymin': 57, 'xmax': 309, 'ymax': 371}}
	{'score': 0.989, 'label': 'dog', 'box': {'xmin': 279, 'ymin': 20, 'xmax': 482, 'ymax': 416}}
SenseTime/deformable-detr:
	{'score': 0.6333, 'label': 'cat', 'box': {'xmin': 78, 'ymin': 60, 'xmax': 309, 'ymax': 372}}
